In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'e:\\project\\Customer Purchase Prediction'

In [ ]:
import dagshub
dagshub.init(repo_owner='PhamAnhTienn', repo_name='Sleep-Efficiency-App', mlflow=True)

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [ ]:
from src.utils import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(Path("config.yml"))
        self.params = read_yaml(Path("params.yml"))
        self.schema = read_yaml(Path("schema.yml"))

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/PhamAnhTienn/Sleep-Efficiency-App.mlflow"
        )

        return model_evaluation_config

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from src.exception import CustomException
import joblib
import datetime

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def evaluation(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        experiment_name = f"xgb_{datetime.now().strftime('%Y%m%d%H%M%S')}"
        mlflow.set_experiment(experiment_name)

        with mlflow.start_run() as run:
            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Save metrics, params, and model to MLflow
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information
                mlflow.xgboost.log_model(model, "model", registered_model_name="sleep_efficiency_model")
            else:
                mlflow.xgboost.log_model(model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluation()
except Exception as e:
    raise CustomException(e, sys)